# GPT-2 XL Text Generation

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
print(torch.__version__)

In [ ]:
from transformers import GPT2Config
from transformers import GPT2LMHeadModel, GPT2Tokenizer

from tqdm import trange


In [ ]:


def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (batch size x vocabulary size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        # scatter sorted tensors to original indexing
        indices_to_remove = sorted_indices_to_remove.scatter(dim=1, index=sorted_indices, src=sorted_indices_to_remove)
        logits[indices_to_remove] = filter_value
    return logits


def sample_sequence(model, length, context, num_samples=1, temperature=1, top_k=0, top_p=0.0,
                    repetition_penalty=1.0, device='cpu'):
    context = torch.tensor(context, dtype=torch.long, device=device)
    context = context.unsqueeze(0).repeat(num_samples, 1)
    generated = context
    with torch.no_grad():
        #for _ in trange(length):
        for _ in range(length):

            inputs = {'input_ids': generated}

            outputs = model(**inputs)  # Note: we could also use 'past' with GPT-2/Transfo-XL/XLNet/CTRL (cached hidden-states)
            next_token_logits = outputs[0][:, -1, :] / (temperature if temperature > 0 else 1.)

            # repetition penalty from CTRL (https://arxiv.org/abs/1909.05858)
            for i in range(num_samples):
                for _ in set(generated[i].tolist()):
                    next_token_logits[i, _] /= repetition_penalty
                
            filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
            if temperature == 0: # greedy sampling:
                next_token = torch.argmax(filtered_logits, dim=-1).unsqueeze(-1)
            else:
                next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_token), dim=1)
    return generated

In [ ]:
#tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
#model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-xl")
model = GPT2LMHeadModel.from_pretrained("gpt2-xl")
#tokenizer = GPT2Tokenizer.from_pretrained("/media/SSD/OPENGPT-2/nips-papers/output")
#model = GPT2LMHeadModel.from_pretrained("/media/SSD/OPENGPT-2/nips-papers/output")

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
  model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"));

In [ ]:
model.eval();

In [ ]:
while True:
    print("\n\n")
    raw_text = input("Model prompt >>> ")
    context_tokens = tokenizer.encode(raw_text, add_special_tokens=False)
    out = sample_sequence(
        model=model,
        context=context_tokens,
        num_samples=1,
        length=200,
        temperature=1.0,
        top_k=0,
        top_p=0.9,
        repetition_penalty=1.0,
        device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        )
    out = out[:, len(context_tokens):].tolist()
    for o in out:
        text = tokenizer.decode(o, clean_up_tokenization_spaces=True)
        #text = text[: text.find(args.stop_token) if args.stop_token else None]
        text = text[: None]

    print("\n\n AI output:\n",text)

# Examples


**Model prompt >>> My dad is an expert at growing orchids.  He has a special room with high humidity to grow them.  Some of the most


 **AI output:**
  impressive orchids I've seen are grown in his room in Texas.  His goal is to grow hundreds of plants from these plants with a few left over in his storage.  He explains that before, it wasn't as easy to grow these plant, but as the decades have gone by, new technologies have come to help grow these plants.  He gets a dozen plants from one single orchid and has to buy so much fertilizer.

Unlike many orchids, lilacs have upright (not hanging) leaves.  For this reason they don't need as much moisture and do a better job of growing faster.   He plants from cuttings and has taken off quite a few years since I was a kid.   He first began planting from cuttings when I was 3.  It was also when I was a teenager that I started experimenting with flowers.  I did only have a punnet tube and used cuttings I had left over from the previous year




**Model prompt >>> The scientists studied the dancing chikens, goats, and pigs and developed a theory that they published in the journal Nature.


 **AI output:**
  That theory states that a brain area called the cerebral cortex directs the brain's dance movements.

"What we believe is that the brains of rodents and birds and probably a host of other creatures are utilizing their own dance patterns as an 'automatic pilot,'" said lead author Dr. John Cacioppo.

In this case, that means that the muscles are doing the driving. The brain receives a signal telling it that it is time to dance. The motor cortex then takes over, activating muscle fibers to propel the animal, and slowly stepping its way through the gaps between movements.

The study says that it is only when animals have worked out what movements they are using that they are able to link movements with speech.

"If you wanted to dance, it would be extremely hard to copy your movements," said Dr. Paul Turkalo, a neuroscientist who studies the structure and function of the human brain at the State University of New York in Buffalo. "You'd have

**Model prompt >>> What time is it?


 **AI output:**
  That confusion @-@ high will make all the difference in an early season episode ; the boss will have to wake up and figure out how to get them all right. " He continued, " It was a show we wanted to do in Season 6, so we were thinking what if we didn 't [ break out ]? How could we let them form and do all this weird little fly through the air. "
 
 = = Relationship = = 
 
 In the season premiere, that would become the romance between Jean and Raven with Scully. While moving into the hospital when she was wounded, Jean finds her love's most treasured memory, Scully tells her not to mind. However, while at the same time questioning Raven, Jean ( who is also the Beatrix of'hit series ) is attracted to a patient. In one scene, she convinces Dr. Baderik to give her a flat to sleep on at night. Raven later revealed to him that she



**Model prompt >>> In September 2010 , a teaser website was


 **AI output:**
  developed that called itself the The Next Key of the Moon, with the release of the comic at the beginning of November 2012. The campaign continues with a live stream of the upcoming events @-@ featuring the entire Moon with a portion of it aired on On Xtra. As of April 2014, The Next Key of the Moon remains in development, and features fifty million extra viewers. 
 
 = = Merchandise = = 
 
 Later in 2011, a teaser webisode ran for the premiere of the series. This included a special feature where they shared on with merchandise who had joined the cult. 
 In February 2012, an exclusive promotional video released for Star Trek : The Next Generation was released, featuring the words " We love the show, you know, don 't you just want to be there " and the musical " You Got Me Back in the Night ". As part of that release, NBC released new series episodes of the series featuring Dr. McCoy, and on



**Model prompt >>> I remeber the day my Dad made kool aid with flour instead of sugar.


 **AI output:**
  I was only seven years old. But I remember every part of that day.

So there's always this strange thing going on. We've been trying to tell you about it all of this time.

We didn't want you to miss the big time. Now I have the chance to tell you how that big time came to be. A year ago, all of you had hoped and prayed that life would finally catch up with our baby. The baby that you expected to see today.

But today… we're telling you. Because you have waited for this, too. But only for a couple weeks. The only way we can hold you back is for this to be here with you for the rest of your life.

Imagine us showing you to the stand at Popeye's. On a white sand beach, ten feet high. That's where we took you all those years ago. Now I'm holding you again. Inside the heart of a magnificent chicken.





